## Capturing Sentences Relating to Ingroup and Outgroup

In this notebook we iterate over all the sentence in the speech if appropriate manually classify each sentence as either ingroup elevation or outgroup othering.

In [ ]:
from spacy import displacy
import os

# dictionary object for the sentences from each file
sentences = dict()

# iterate over sentences from each orator, remove any return symbols and add to dictionary object
# note, sentences are identified by their index in a document rather than the word
for sentence in doc.sents:
    if doc[sentence.end -1].text == '\n':
        sentences[len(sentences)] = [sentence.start, sentence.end - 1]
    else:
        sentences[len(sentences)] = [sentence.start, sentence.end]
    
# print the first five sentences of each sentence dictionary
i=0
for key, value in sentences.items():
    print(key, '=>', doc[value[0]:value[1]])
    i+=1
    if i == 5:
        break
print()

ingroup = dict()
outgroup = dict()
index = dict()

# open previous file and progress index

filepath = 'C:/Users/Steve/OneDrive - University of Southampton/CulturalViolence/KnowledgeBases/'

try:
    with open(os.path.join(filepath, "ingroup_sentences.json"), 'r') as fp:
        ingroup = json.load(fp)
except:
    pass

try:
    with open(os.path.join(filepath, "outgroup_sentences.json"), 'r') as fp:
        outgroup = json.load(fp)
except:
    pass

try:
    with open(os.path.join(filepath, "index.json"), 'r') as fp:
        index = json.load(fp)
except:
    index["index"] = 0


print(index)

#iterate over each sentence dictionary for classification of ingroup or outgroup
while i < len(sentences):
    # get the current progress through the dictionary object
    i = index["index"]
    
    # record progress  through dictionary object
    with open(os.path.join(filepath, "index.json"), "wb") as f:
            f.write(json.dumps(index).encode("utf-8"))
    
    # print current sentence
    print('-----')
    print('index: ', i)
    displacy.render(doc[sentences[i][0]:sentences[i][1]], style = 'ent')
    print(i, '/', len(sentences), '=>', doc[sentences[i][0]:sentences[i][1]])
    entry = input('ingroup(i) / outgroup(o) / back(b)').lower()
    
    # ask if sentence is refering to an ingroup or outgroup
    if entry in ['i', 'o']:        
        if entry == 'i': # add sentence to ingroup dictionary if user selects ingroup
            print(len(ingroup), ' => ingroup add: ', doc[sentences[i][0]:sentences[i][1]])
            ingroup[len(ingroup)] = [sentences[i][0], sentences[i][1]]
            
            # write dictionary to file
            with open(os.path.join(filepath, "ingroup_sentences.json"), "wb") as f:
                f.write(json.dumps(ingroup).encode("utf-8"))
            
        else: # else add sentence to outgroup dictionary
            print(len(outgroup), ' => outgroup add: ', doc[sentences[i][0]:sentences[i][1]])
            outgroup[len(outgroup)] = [sentences[i][0], sentences[i][1]]
            
            # write dictionary to file
            with open(os.path.join(filepath, "outgroup_sentences.json"), "wb") as f:
                f.write(json.dumps(outgroup).encode("utf-8"))
                
        # increase index by 1
        index["index"] += 1
    
    
    # if user enters 'b' then go back by 1 in the dictionary and delete
    elif entry == 'b': 
        if i != 0:
            
            # test whether the previous sentence was ingroup or outgroup and delete from respective dictionary
            
            if len(ingroup) - 1 >= 0 and sentences[i-1] == ingroup[len(ingroup) - 1]:
                print('deleting from ingroup: ', doc[ingroup[len(ingroup) - 1][0]:ingroup[len(ingroup) - 1][1]])
                del(ingroup[len(ingroup) - 1])

                with open(os.path.join(filepath, "ingroup_sentences.json"), "wb") as f:
                    f.write(json.dumps(ingroup).encode("utf-8"))

            elif len(outgroup) - 1 >= 0 and sentences[i-1] == outgroup[len(outgroup) - 1]:
                print('deleting from outgroup: ', doc[outgroup[len(outgroup) - 1][0]:outgroup[len(outgroup) - 1][1]])
                del(outgroup[len(outgroup) - 1])

                with open(os.path.join(filepath, "outgroup_sentences.json"), "wb") as f:
                    f.write(json.dumps(outgroup).encode("utf-8"))

            index["index"] -= 1
        
        else:
            print('iterating backwards by one sentence')
            pass
        
    # quit    
    elif entry == 'q':
        break
        
    else:
        index["index"] += 1

print(f'completed at {str(datetime.datetime.now())}') #1220